[View in Colaboratory](https://colab.research.google.com/github/jagatfx/turicreate-colab/blob/master/turicreate_activity_classification.ipynb)

# Activity Classification
https://apple.github.io/turicreate/docs/userguide/activity_classifier/

Activity classification is the task of identifying a pre-defined set of physical actions using motion-sensory inputs. Such sensors include accelerometers, gyroscopes, thermostats, and more found in most handheld devices today.

Possible applications include counting swimming laps using a watch's accelerometer data, turning on Bluetooth controlled lights when recognizing a certain gesture using gyroscope data from a handheld phone, or creating shortcuts to your favorite phone applications using hand gestures.

The activity classifier in Turi Create creates a deep learning model capable of detecting temporal features in sensor data, lending itself well to the task of activity classification. Before we dive into the model architecture, let's see a working example.

## Turi Create and GPU Setup

In [0]:
!apt install libnvrtc8.0
!pip uninstall -y mxnet-cu80 && pip install mxnet-cu80==1.1.0
!pip install turicreate

## Google Drive Access

You will be asked to click a link to generate a secret key to access your Google Drive. 

Copy and paste secret key it into the space provided with the notebook.

In [1]:
import os.path
from google.colab import drive

# mount Google Drive to /content/drive/My Drive/
if os.path.isdir("/content/drive/My Drive"):
  print("Google Drive already mounted")
else:
  drive.mount('/content/drive')

Google Drive already mounted


## Fetch Data

In [0]:
import os.path
import urllib.request
import tarfile
import zipfile
import gzip
from shutil import copy

def fetch_remote_datafile(filename, remote_url):
  if os.path.isfile("./" + filename):
    print("already have " + filename + " in workspace")
    return
  print("fetching " + filename + " from " + remote_url + "...")
  urllib.request.urlretrieve(remote_url, "./" + filename)

def cache_datafile_in_drive(filename):
  if os.path.isfile("./" + filename) == False:
    print("cannot cache " + filename + ", it is not in workspace")
    return
  
  data_drive_path = "/content/drive/My Drive/Colab Notebooks/data/"
  if os.path.isfile(data_drive_path + filename):
    print("" + filename + " has already been stored in Google Drive")
  else:
    print("copying " + filename + " to " + data_drive_path)
    copy("./" + filename, data_drive_path)
  

def load_datafile_from_drive(filename, remote_url=None):
  data_drive_path = "/content/drive/My Drive/Colab Notebooks/data/"
  if os.path.isfile("./" + filename):
    print("already have " + filename + " in workspace")
  elif os.path.isfile(data_drive_path + filename):
    print("have " + filename + " in Google Drive, copying to workspace...")
    copy(data_drive_path + filename, ".")
  elif remote_url != None:
    fetch_remote_datafile(filename, remote_url)
  else:
    print("error: you need to manually download " + filename + " and put in drive")
    
def extract_datafile(filename, expected_extract_artifact=None):
  if expected_extract_artifact != None and (os.path.isfile(expected_extract_artifact) or os.path.isdir(expected_extract_artifact)):
    print("files in " + filename + " have already been extracted")
  elif os.path.isfile("./" + filename) == False:
    print("error: cannot extract " + filename + ", it is not in the workspace")
  else:
    extension = filename.split('.')[-1]
    if extension == "zip":
      print("extracting " + filename + "...")
      data_file = open(filename, "rb")
      z = zipfile.ZipFile(data_file)
      for name in z.namelist():
          print("    extracting file", name)
          z.extract(name, "./")
      data_file.close()
    elif extension == "gz":
      print("extracting " + filename + "...")
      if filename.split('.')[-2] == "tar":
        tar = tarfile.open(filename)
        tar.extractall()
        tar.close()
      else:
        data_zip_file = gzip.GzipFile(filename, 'rb')
        data = data_zip_file.read()
        data_zip_file.close()
        extracted_file = open('.'.join(filename.split('.')[0:-1]), 'wb')
        extracted_file.write(data)
        extracted_file.close()
    elif extension == "tar":
      print("extracting " + filename + "...")
      tar = tarfile.open(filename)
      tar.extractall()
      tar.close()
    elif extension == "csv":
      print("do not need to extract csv")
    else:
      print("cannot extract " + filename)
      
def load_cache_extract_datafile(filename, expected_extract_artifact=None, remote_url=None):
  load_datafile_from_drive(filename, remote_url)
  extract_datafile(filename, expected_extract_artifact)
  cache_datafile_in_drive(filename)
  

In [5]:
load_cache_extract_datafile("HAPT Data Set.zip", "RawData", "http://archive.ics.uci.edu/ml/machine-learning-databases/00341/HAPT%20Data%20Set.zip")

already have HAPT Data Set.zip in workspace
files in HAPT Data Set.zip have already been extracted
HAPT Data Set.zip has already been stored in Google Drive


## Setup Turi Create

In [0]:
import mxnet as mx
import turicreate as tc

In [0]:
# Use all GPUs (default)
tc.config.set_num_gpus(-1)

# Use only 1 GPU
#tc.config.set_num_gpus(1)

# Use CPU
#tc.config.set_num_gpus(0)

## Data Preparation

https://apple.github.io/turicreate/docs/userguide/activity_classifier/data-preparation.html

In [11]:
data_dir = './RawData/'

def find_label_for_containing_interval(intervals, index):
    containing_interval = intervals[:, 0][(intervals[:, 1] <= index) & (index <= intervals[:, 2])]
    if len(containing_interval) == 1:
        return containing_interval[0]

# Load labels
labels = tc.SFrame.read_csv(data_dir + 'labels.txt', delimiter=' ', header=False, verbose=False)
labels = labels.rename({'X1': 'exp_id', 'X2': 'user_id', 'X3': 'activity_id', 'X4': 'start', 'X5': 'end'})
labels.head()

exp_id,user_id,activity_id,start,end
1,1,5,250,1232
1,1,7,1233,1392
1,1,4,1393,2194
1,1,8,2195,2359
1,1,5,2360,3374
1,1,11,3375,3662
1,1,6,3663,4538
1,1,10,4539,4735
1,1,4,4736,5667
1,1,9,5668,5859


Next, we need to get the accelerometer and gyroscope data for each experiment. For each experiment, every sensor's data is in a separate file. In the code below we load the accelerometer and gyroscope data from all experiments into a single SFrame. While loading the collected samples, we also calculate the label for each sample using our previously defined function. The final SFrame contains a column named exp_id to identify each unique sessions.

In [0]:
from glob import glob

acc_files = glob(data_dir + 'acc_*.txt')
gyro_files = glob(data_dir + 'gyro_*.txt')

# Load data
data = tc.SFrame()
files = zip(sorted(acc_files), sorted(gyro_files))
for acc_file, gyro_file in files:
    exp_id = int(acc_file.split('_')[1][-2:])

    # Load accel data
    sf = tc.SFrame.read_csv(acc_file, delimiter=' ', header=False, verbose=False)
    sf = sf.rename({'X1': 'acc_x', 'X2': 'acc_y', 'X3': 'acc_z'})
    sf['exp_id'] = exp_id

    # Load gyro data
    gyro_sf = tc.SFrame.read_csv(gyro_file, delimiter=' ', header=False, verbose=False)
    gyro_sf = gyro_sf.rename({'X1': 'gyro_x', 'X2': 'gyro_y', 'X3': 'gyro_z'})
    sf = sf.add_columns(gyro_sf)

    # Calc labels
    exp_labels = labels[labels['exp_id'] == exp_id][['activity_id', 'start', 'end']].to_numpy()
    sf = sf.add_row_number()
    sf['activity_id'] = sf['id'].apply(lambda x: find_label_for_containing_interval(exp_labels, x))
    sf = sf.remove_columns(['id'])

    data = data.append(sf)

Finally, we encode the labels back into a readable string format, and save the resulting SFrame.

In [0]:
target_map = {
    1.: 'walking',          
    2.: 'climbing_upstairs',
    3.: 'climbing_downstairs',
    4.: 'sitting',
    5.: 'standing',
    6.: 'laying'
}

# Use the same labels used in the experiment
data = data.filter_by(list(target_map.keys()), 'activity_id')
data['activity'] = data['activity_id'].apply(lambda x: target_map[x])
data = data.remove_column('activity_id')

data.save('hapt_data.sframe')

In [15]:
data.head()

acc_x,acc_y,acc_z,exp_id,gyro_x
1.020833394742025,-0.1250000020616516,0.105555564319952,1,-0.002748893573880196
1.025000070391787,-0.1250000020616516,0.1013888947481719,1,-0.0003054326225537807
1.020833394742025,-0.1250000020616516,0.1041666724366978,1,0.01221730466932058
1.016666719092262,-0.1250000020616516,0.1083333359304957,1,0.01130100712180138
1.018055610975516,-0.1277777858281599,0.1083333359304957,1,0.01099557429552078
1.018055610975516,-0.1291666655554495,0.1041666724366978,1,0.009162978269159794
1.01944450285877,-0.1250000020616516,0.1013888947481719,1,0.01007927674800158
1.016666719092262,-0.1236111101783975,0.09722222517639174,1,0.01374446786940098
1.020833394742025,-0.1277777858281599,0.09861111705964588,1,0.009773843921720982
1.01944450285877,-0.1152777831908018,0.09444444748786576,1,0.01649336144328117


In [16]:
data.groupby('activity', [tc.aggregate.COUNT]).sort("Count", ascending = False)

activity,Count
standing,138105
laying,136865
sitting,126677
walking,122091
climbing_upstairs,116707
climbing_downstairs,107961


## Example Activity Classififcation - HAPT Data

In [0]:
# Load sessions from preprocessed data
data = tc.SFrame('hapt_data.sframe')

In [0]:
# Train/test split by recording sessions
train, test = tc.activity_classifier.util.random_split_by_session(data, session_id='exp_id', fraction=0.8)

In [7]:
# Create an activity classifier
model = tc.activity_classifier.create(train, session_id='exp_id', target='activity', prediction_window=50)

The dataset has less than the minimum of 100 sessions required for train-validation split. Continuing without validation set


Pre-processing 585143 samples...

Using sequences of size 1000 for model creation.

Processed a total of 48 sessions.

Using GPU to create model (CUDA)
+----------------+----------------+----------------+----------------+
| Iteration      | Train Accuracy | Train Loss     | Elapsed Time   |
+----------------+----------------+----------------+----------------+
| 1              | 0.623          | 0.977          | 0.6            |
| 2              | 0.810          | 0.541          | 1.2            |
| 3              | 0.846          | 0.412          | 1.8            |
| 4              | 0.863          | 0.359          | 2.4            |
| 5              | 0.873          | 0.322          | 3.0            |
| 6              | 0.889          | 0.293          | 3.6            |
| 7              | 0.895          | 0.264          | 4.2            |
| 8              | 0.902          | 0.242          | 4.8            |
| 9              | 0.911          | 0.224          | 5.4            |
| 10             | 0.916          | 0.208          | 6.0            |
+----------------+----------------+----------------+-----

In [17]:
# Evaluate the model and save the results into a dictionary
metrics = model.evaluate(test)
print(metrics)

{'accuracy': 0.9324280455461433, 'auc': 0.994000145914642, 'precision': 0.9344077064028339, 'recall': 0.9313875060212965, 'f1_score': 0.9325884730108659, 'log_loss': 0.2183073822297761, 'confusion_matrix': Columns:
	target_label	str
	predicted_label	str
	count	int

Rows: 31

Data:
+---------------------+---------------------+-------+
|     target_label    |   predicted_label   | count |
+---------------------+---------------------+-------+
| climbing_downstairs |  climbing_upstairs  |  640  |
|  climbing_upstairs  |  climbing_upstairs  | 23073 |
|  climbing_upstairs  | climbing_downstairs |  1048 |
|        laying       |       walking       |   31  |
| climbing_downstairs | climbing_downstairs | 21240 |
|       sitting       |       standing      |  3346 |
|        laying       |        laying       | 29554 |
|       standing      |       standing      | 29827 |
|       walking       |       sitting       |  1351 |
|       sitting       |       sitting       | 25134 |
+---------------

In [9]:
print(metrics['accuracy'])

0.9324280455461433


Since we have created the model with samples taken at 50Hz and set the prediction_window to 50, we will get one prediction per second. Invoking our newly created model on the above 3-seconds walking example produces the following per-second predictions:

In [12]:
walking_3_sec = data[(data['activity'] == 'walking') & (data['exp_id'] == 1)][1000:1150]
model.predict(walking_3_sec, output_frequency='per_window')

prediction_id,exp_id,class
0,1,walking
1,1,walking
2,1,walking


## Save / Export Model

In [0]:
# Save the model for later use in Turi Create
model.save('ActivityClassifier.model')

In [11]:
# Export for use in Core ML
model.export_coreml('ActivityClassifier.mlmodel')

/usr/local/lib/python3.6/dist-packages/coremltools/_deps/__init__.py:118: DeprecationWarning: The 'warn' function is deprecated, use 'warning' instead
  % (tensorflow.__version__, TF_MAX_VERSION))


In [0]:
# download mlmodel locally
from google.colab import files
files.download("ActivityClassifier.mlmodel")

In [13]:
# copy model to Google Drive
from shutil import copy
copy("/content/ActivityClassifier.mlmodel", "/content/drive/My Drive/Colab Notebooks/data/models/ActivityClassifier.mlmodel")

'/content/drive/Colab Notebooks/data/models/ActivityClassifier.mlmodel'

In [15]:
# copy model to Google Drive
from shutil import copytree
copytree("/content/ActivityClassifier.model", "/content/drive/My Drive/Colab Notebooks/data/models/ActivityClassifier.model")

'/content/drive/Colab Notebooks/data/models/ActivityClassifier.model'

## How does this work?

The deep learning model relies on convolutional layers to extract temporal features from a single prediction window, for example an arching movement could possibly be a strong indicator of swimming. Furthermore, it relies on recurrent layers to extract temporal features over time, for example if a subject was swimming in the previous timestamp, then it is most likely not sky diving in the next. Below is a sketch of the neural network used for the activity classifier in Turi Create.

![deep learning model](https://apple.github.io/turicreate/docs/userguide/activity_classifier/images/activity_classifier_network.png)

A single input to the neural network is a session as defined in the previous section. The convolutional layer operates on each prediction window, finding spatial features that may be relevant to the labeled activities. 

![prediction window](https://apple.github.io/turicreate/docs/userguide/activity_classifier/images/convolutional_filter.png)

The output of the convolutional layer is a vector representation for each prediction window, encoding these learnt features. The recurrent layer takes as input a sequence of these vectors.

The recurrent layer is specialized for learning temporal features across sequences. For example, it may learn that spatial features associated with walking are more likely to occur after detecting spatial features associated with running. These features are further encoded into the output of the recurrent layer.

In order to detect these features along sessions the recurrent layer takes into account it's own state - the output of the recurrent layer for the previous prediction window. The output of the recurrent layer for the current prediction window is turned into a probability vector across all desired activities to produce the final classification.

Blogs:
*   https://medium.com/@howal/activity-monitoring-with-apples-turi-create-machine-learning-1043ce5b9203